In [150]:
import pandas as pd
import numpy as np

from pycel import ExcelCompiler
import time

In [ ]:
# If True, the Excel model is evaluated again, and exported to CSV as cache. If False, cached CSV files are used.
generate_model_cache = True

aircraft_types = ["A333", "A350", "A380"] #, "B789", "B77W", "B748"]

In [151]:
if(generate_model_cache == True):
    model_path = "model/EEAEmissionsCalculator.xlsm"
    model = ExcelCompiler(filename=model_path)

    flight_df_dict = {}
    
    for aircraft_type in aircraft_types:
        # Set settings in Excel model to aircraft type
        model.set_value('Emissions calculator!D17', aircraft_type)

        # Evaluate model for this aircraft type and store results in dictionary

        evaluation_results = model.evaluate('Emissions calculator!D33:P42')

        flight_df_dict[aircraft_type] = 
    
    model.evaluate('Emissions calculator!D33:P42')


C:\Users\marko\anaconda3\envs\LCAero\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Users\marko\anaconda3\envs\LCAero\lib\site-packages\openpyxl\reader\drawings.py:29: UserWarning: DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.
  warn("DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.")


In [6]:
print("D17 is {}".format(model.evaluate('Emissions calculator!D17')))

D17 is A320


In [69]:
model_a320 = model.evaluate('Emissions calculator!D33:P42')

In [38]:
def move_column_inplace(df, col, pos):
    col = df.pop(col)
    df.insert(pos, col.name, col)

    return df

In [130]:
def Convert_Excel_Times(decimal_time):
    hours = decimal_time * 24
    minutes = 60 * hours
    seconds = 60 * minutes

    time_return = time.strftime('%H:%M:%S', time.gmtime(seconds))
    return time_return

In [131]:
def Convert_Column_Times(time_column):
    return_list = []
    for decimal_time in time_column:
        return_list.append(Convert_Excel_Times(decimal_time))

    return return_list

In [132]:
def Convert_Column_Times_to_Format(time_column):
    return_list = []
    for timestamp in time_column:
        return_list.append(time.strftime('%H:%M:%S', time.gmtime(timestamp)))

    return return_list    


In [133]:
def Convert_Column_Times_To_Seconds(time_column):
    return_list = []
    for decimal_time in time_column:
        return_list.append(decimal_time * 24 * 3600)

    return return_list

In [82]:
a320_df = pd.read_csv("model/a320.csv", index_col=0)

# Remove first rows
a320_df = a320_df.iloc[2:]

# Convert times to second
a320_df.insert(loc=0, column='Duration [s]', value=Convert_Column_Times_To_Seconds(a320_df["Duration [Excel]"]))

# Drop other time columns
a320_df = a320_df.drop(columns='Duration [hh:mm:ss]')
a320_df = a320_df.drop(columns='Duration [Excel]')

#a320_df = move_column_inplace(a320_df, "Duration [Excel]", 0)

In [83]:
a320_df.loc[-1] = ['0','0','0','0','0','0','0','0','0','0','0','0','0']  # adding a row
a320_df.index = a320_df.index + 1  # shifting index
a320_df = a320_df.sort_index()  # sorting by index
a320_df = a320_df.apply(pd.to_numeric)


In [84]:
a320_df

,Duration [s],Distance [nm],Cruise flight level [100 ft],Fuel burn [kg],CO2 [kg],NOx [kg],SOx [kg],H2O [kg],CO [kg],HC [kg],PM non volatile [kg],PM volatile [kg],PM total [kg]
0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1296.629435,125,180,931.918255,2935.542504,17.531388,0.782811,1146.259148,3.021782,0.618175,0.012599,0.080212,0.092810
4,1878.498041,200,270,1356.448631,4272.813188,25.704304,1.139418,1668.429455,4.485261,0.920408,0.020747,0.126463,0.147210
5,2264.154874,250,280,1647.380366,5189.248151,30.111806,1.383800,2026.279042,5.003805,1.032863,0.023276,0.159240,0.182516
6,4249.327158,500,320,2946.002057,9279.906480,47.543313,2.474647,3623.584878,7.361388,1.547205,0.032108,0.319169,0.351278
7,6304.618151,750,360,4124.493311,12992.153930,62.109528,3.464575,5073.122013,9.790558,2.077127,0.040695,0.484610,0.525305
8,8316.562047,1000,380,5273.370305,16611.116461,76.675639,4.429635,6486.254233,12.331605,2.636816,0.053192,0.670758,0.723950
9,12345.142691,1500,380,7768.611924,24471.127560,108.797639,6.525626,9555.393114,16.709608,3.609605,0.072606,1.019896,1.092502
10,16366.728731,2000,380,10483.841488,33024.100686,144.828372,8.806416,12895.093425,20.694100,4.494982,0.087194,1.340740,1.427934
11,20390.277761,2500,380,12914.240729,40679.858297,175.230023,10.847923,15884.439324,25.012619,5.454524,0.104770,1.685642,1.790412


In [145]:
def Fetch_Rows_For_Interpolation(flight_df, time_given):
    """Fetches the two rows in the data next to the given time (closest lower than given time and closest higher than given time), used for linear interpolation"""
    
    df_mask = flight_df['Duration [s]'] <= time_given

    # Get last row where time is lower than given time
    flight_df_lower_bound = flight_df[df_mask].iloc[-1:]
    
    index_lower_bound = flight_df_lower_bound.iloc[0:].index
    
    # Look up position of index in non-filtered dataframe
    loc_lower_bound = flight_df.index.get_loc(index_lower_bound[0])

    # Upper bound is one position (row) higher in Dataframe
    loc_upper_bound = loc_lower_bound + 1

    flight_df_upper_bound = flight_df.iloc[[loc_upper_bound]]

    df_list = [flight_df_lower_bound, flight_df_upper_bound]
    flight_df_interval = pd.concat(df_list)

    return flight_df_interval

In [147]:
def Interpolate_Emissions(flight_df, given_times):
    """Receives list of time points for which emissions are to be interpolated
    :return: Dataframe with emissions for each time point
    """

    df_list = []

    for given_time in given_times:

        # Get data higher and lower than given time
        flight_df_interval = Fetch_Rows_For_Interpolation(flight_df, given_time)

        # Init dictionary
        emissions_at_time = {}

        x = given_time

        # Iterate through columns to interpolate each of them individually
        for (column_name, column_data) in flight_df_interval.iteritems():

            # Get x values (times) as interpolation boundaries on x-axis
            if(column_name == "Duration [s]"):
                x1 = flight_df_interval["Duration [s]"].iloc[0]
                x2 = flight_df_interval["Duration [s]"].iloc[1]

            y1 = column_data.iloc[0]
            y2 = column_data.iloc[1]

            # Linear interpolation
            y = y1 + (x - x1)*((y2-y1)/(x2-x1))

            # Store interpolated column value to dictionary
            emissions_at_time[column_name] = [y]

        emissions_at_time_df = pd.DataFrame.from_dict(emissions_at_time)
        emissions_at_time_df.insert(loc=1, column='Duration [hh:mm:ss]', value=Convert_Column_Times_to_Format(emissions_at_time_df["Duration [s]"]))

        df_list.append(emissions_at_time_df)
    
    emissions_at_times_df = pd.concat(df_list)

    return emissions_at_times_df


In [148]:
desired_times = [100, 2000, 3000]

emissions_at_times_output_df = Interpolate_Emissions(a320_df, desired_times)

In [149]:
emissions_at_times_output_df

,Duration [s],Duration [hh:mm:ss],Distance [nm],Cruise flight level [100 ft],Fuel burn [kg],CO2 [kg],NOx [kg],SOx [kg],H2O [kg],CO [kg],HC [kg],PM non volatile [kg],PM volatile [kg],PM total [kg]
0,100.0,00:01:40,9.640380,13.882147,71.872366,226.397953,1.352074,0.060373,88.402987,0.233049,0.047676,0.000972,0.006186,0.007158
0,2000.0,00:33:20,215.752600,273.150520,1448.107258,4561.537862,27.092896,1.216411,1781.170685,4.648629,0.955837,0.021544,0.136790,0.158333
0,3000.0,00:50:00,342.667666,294.826827,2128.741328,6705.535184,36.573154,1.788145,2618.353454,5.877691,1.223514,0.026550,0.218521,0.245071


# Notes

Manipulating the EEA Aviation Emissions Calculator via the Python library PyCel is slow. Reading the "Climb, cruise, descent" stage length (NM) information from the Excel model for a flight with an Airbus A320 takes about 3.5 minutes.

The primary goal of this project is not the development of a Python implementation of the EEA Aviation Emissions Calculator. Therefore, we do not focus on increasing the excecution speed of the Excel model in Python. Instead, we cache the information. The model is executed in Excel with standard settings for each aircraft that should be covered by LCAero. The results (emissions) are cached in a CSV file that can be imported by a Python script without analyzing the spreadsheet again.

- Add simple explaination of EEA model to report, explain different flight phases, etc.
- Define LTO and CCD stage length